In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.auto import tqdm
import os
from PIL import Image
import sys

In [23]:
def is_valid_image(filepath):
    try:
        with Image.open(filepath) as img:
            img.verify()
        return True
    except:
        print(f"Corrupted Image Found: {filepath}", file=sys.stderr)
        return False

class FilteredImageFolder(datasets.ImageFolder):
    def __init__(self, root, transform=None):
        super(FilteredImageFolder, self).__init__(root=root, transform = transform)
        valid_images = []

        for item in self.imgs:
            if is_valid_image(item[0]):
                valid_images.append(item)
        self.imgs = valid_images
        self.samples = valid_images

In [24]:
img_width, img_height = 150, 150
batch_size = 32
num_epochs = 20

train_transform = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.RandomRotation(40),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(0, translate=(0.2,0.2), scale=(0.8, 1.2), shear=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456,0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456,0.406], [0.229, 0.224, 0.225])
])

In [25]:
print("Loading and Validating training dataset...")
train_dataset = FilteredImageFolder(
    root='dataset/train',
    transform=train_transform
)

print("Loading and validating validation dataset...")
val_dataset = FilteredImageFolder(
    root='dataset/validation',
    transform=val_transforms
)

print("Training classes:", train_dataset.classes)
print("Validation classes:", val_dataset.classes)

Loading and Validating training dataset...


Corrupted Image Found: dataset/train\dogs\11702.jpg


Loading and validating validation dataset...
Training classes: ['cats', 'dogs']
Validation classes: ['cats', 'dogs']


In [26]:
train_loader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers=0,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers = 0,
    pin_memory=True
)


# Building CNN

In [27]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,32,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(32,64,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(64,128,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128,128,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128,512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avg_pool(x)
        x = self.classifier(x)
        return x

In [28]:
model = CNNClassifier(num_classes= len(train_dataset.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
train_losses = []
val_losses = []

# Modified training loop to track losses
best_val_acc = 0.0

for epoch in range(num_epochs):
    print(f'\nEpoch [{epoch+1}/{num_epochs}]')
    
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    pbar = tqdm(total=len(train_loader), desc=f'Training')
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs, labels
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        grad_norms = {}
        for name, param in model.named_parameters():
            if param.grad is not None:
                grad_norms[name] = param.grad.norm().item()
        
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        
        avg_loss = train_loss / (i + 1)
        acc = 100. * train_correct / train_total
        pbar.set_description(f'Train | Loss: {avg_loss:.4f} | Acc: {acc:.2f}%')
        pbar.update(1)
    
    pbar.close()
    
    # Store average training loss
    train_losses.append(train_loss / len(train_loader))
    
    print("\nGradient Norms:")
    for name, norm in grad_norms.items():
        print(f"{name}: {norm:.4f}")

        # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    pbar = tqdm(total=len(val_loader), desc=f'Validation')
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            inputs, labels = inputs, labels
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
            
            avg_loss = val_loss / (i + 1)
            acc = 100. * val_correct / val_total
            pbar.set_description(f'Val | Loss: {avg_loss:.4f} | Acc: {acc:.2f}%')
            pbar.update(1)
    
    pbar.close()

    # Store average validation loss
    val_losses.append(val_loss / len(val_loader))
    
    train_acc = 100. * train_correct / train_total
    val_acc = 100. * val_correct / val_total
    
    print(f'\nEpoch Summary:')
    print(f'Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc:.2f}%')
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'catsanddogs_classifier.pth')
        print(f'New best model saved! (Validation Accuracy: {val_acc:.2f}%)')

        


Epoch [1/20]


Training:   0%|          | 0/298 [00:00<?, ?it/s]

E:\ai-ml-learning\streamlit\environment\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Gradient Norms:
features.0.weight: 0.7935
features.0.bias: 0.1984
features.3.weight: 0.9426
features.3.bias: 0.2497
features.6.weight: 0.4788
features.6.bias: 0.2531
features.9.weight: 0.2229
features.9.bias: 0.1569
classifier.1.weight: 0.1418
classifier.1.bias: 0.1957
classifier.3.weight: 0.4177
classifier.3.bias: 0.2848


Validation:   0%|          | 0/34 [00:00<?, ?it/s]

...

Training:   0%|          | 0/298 [00:00<?, ?it/s]


Gradient Norms:
features.0.weight: 0.2231
features.0.bias: 0.0827
features.3.weight: 0.2245
features.3.bias: 0.0531
features.6.weight: 0.0726
features.6.bias: 0.0728
features.9.weight: 0.0410
features.9.bias: 0.0388
classifier.1.weight: 0.0461
classifier.1.bias: 0.0494
classifier.3.weight: 0.0474
classifier.3.bias: 0.0060


Validation:   0%|          | 0/34 [00:00<?, ?it/s]


Epoch Summary:
Train Loss: 0.6360, Train Acc: 63.72%
Val Loss: 0.6209, Val Acc: 66.33%
New best model saved! (Validation Accuracy: 66.33%)

Epoch [3/20]


Training:   0%|          | 0/298 [00:00<?, ?it/s]